**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
ASSAY=Tewhey_K562_TileMPRA
PREFIX=Tewhey_K562_TileMPRA
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
ls ${FD_INP}

OL13_20220512.hg19.norm.Input.mean.stranded_pos.bed
OL13_20220512.hg19.norm.Input.rep1.stranded_pos.bed
OL13_20220512.hg19.norm.Input.rep2.stranded_pos.bed
OL13_20220512.hg19.norm.Input.rep3.stranded_pos.bed
OL13_20220512.hg19.norm.Input.rep4.stranded_pos.bed
OL13_20220512.hg19.norm.Log2FC.mean.stranded_pos.bed
OL13_20220512.hg19.norm.Output.mean.stranded_pos.bed
OL13_20220512.hg19.norm.Output.rep1.stranded_pos.bed
OL13_20220512.hg19.norm.Output.rep2.stranded_pos.bed
OL13_20220512.hg19.norm.Output.rep3.stranded_pos.bed
OL13_20220512.hg19.norm.Output.rep4.stranded_pos.bed
OL13_20220512.hg19.raw.Input.rep1.stranded_pos.bed
OL13_20220512.hg19.raw.Input.rep2.stranded_pos.bed
OL13_20220512.hg19.raw.Input.rep3.stranded_pos.bed
OL13_20220512.hg19.raw.Input.rep4.stranded_pos.bed
OL13_20220512.hg19.raw.Output.rep1.stranded_pos.bed
OL13_20220512.hg19.raw.Output.rep2.stranded_pos.bed
OL13_20220512.hg19.raw.Output.rep3.stranded_pos.bed
OL13_20220512.hg19.raw.Output.rep4.stranded_pos.bed
OL13_20220

In [3]:
ASSAY=Tewhey_K562_TileMPRA
PREFIX=Tewhey_K562_TileMPRA
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FP_INPS=($(ls ${FD_INP}/*hg38*Input*) $(ls ${FD_INP}/*hg38*Output*))

echo "${#FP_INPS[@]}"
basename -a ${FP_INPS[@]}

60
OL13_20220512.hg38.norm.Input.mean.stranded_pos.bed
OL13_20220512.hg38.norm.Input.rep1.stranded_pos.bed
OL13_20220512.hg38.norm.Input.rep2.stranded_pos.bed
OL13_20220512.hg38.norm.Input.rep3.stranded_pos.bed
OL13_20220512.hg38.norm.Input.rep4.stranded_pos.bed
OL13_20220512.hg38.raw.Input.rep1.stranded_pos.bed
OL13_20220512.hg38.raw.Input.rep2.stranded_pos.bed
OL13_20220512.hg38.raw.Input.rep3.stranded_pos.bed
OL13_20220512.hg38.raw.Input.rep4.stranded_pos.bed
OL43_20221003.hg38.norm.Input.mean.stranded_pos.bed
OL43_20221003.hg38.norm.Input.rep1.stranded_pos.bed
OL43_20221003.hg38.norm.Input.rep2.stranded_pos.bed
OL43_20221003.hg38.norm.Input.rep3.stranded_pos.bed
OL43_20221003.hg38.norm.Input.rep4.stranded_pos.bed
OL43_20221003.hg38.norm.Input.rep5.stranded_pos.bed
OL43_20221003.hg38.norm.Input.rep6.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep1.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep2.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep3.stranded_pos.bed
OL43_20221003.hg

In [4]:
ASSAY=Tewhey_K562_TileMPRA
PREFIX=Tewhey_K562_TileMPRA
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FP_INPS=($(ls ${FD_INP}/*hg38*Log2FC*))

echo "${#FP_INPS[@]}"
basename -a ${FP_INPS[@]}

3
OL13_20220512.hg38.norm.Log2FC.mean.stranded_pos.bed
OL43_20221003.hg38.norm.Log2FC.mean.stranded_pos.bed
OL45_20220927.hg38.norm.Log2FC.mean.stranded_pos.bed


## Calculate coverage perbase for Log2FC (Mean)

In [5]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/coverage_perbase_TMPRA_log2fc.%a.txt \
    --array 0-2 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=Tewhey_K562_TileMPRA
PREFIX=Tewhey_K562_TileMPRA

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_counts
FD_REG=${FD_RES}/results/region

FP_REG=${FD_REG}/region_screened_selected_perbase.bed.gz
FP_INPS=($(ls ${FD_INP}/*hg38*Log2FC*))

FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP}.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_REG}
echo
echo "show first few lines of file"
zcat ${FP_REG} | head -5
echo
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_INP}
echo
echo "show first few lines of file"
cat ${FP_INP} | head -5
echo

### execute
bedtools map \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -o mean \
| grep -v $'\t\\.' \
| gzip -c \
> ${FP_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of file"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29574566


## Calculate coverage perbase for input and output (Sum)

In [6]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/coverage_perbase_TMPRA_input_output.%a.txt \
    --array 0-59 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=Tewhey_K562_TileMPRA
PREFIX=Tewhey_K562_TileMPRA

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_counts
FD_REG=${FD_RES}/results/region

FP_REG=${FD_REG}/region_screened_selected_perbase.bed.gz
FP_INPS=($(ls ${FD_INP}/*hg38*Input*) $(ls ${FD_INP}/*hg38*Output*))

FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP}.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_REG}
echo
echo "show first few lines of file"
zcat ${FP_REG} | head -5
echo
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_INP}
echo
echo "show first few lines of file"
cat ${FP_INP} | head -5
echo

### execute
bedtools map \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -o sum \
| grep -v $'\t\\.' \
| gzip -c \
> ${FP_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of file"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29574569


## Check results

In [7]:
cat ${FD_LOG}/coverage_perbase_TMPRA_log2fc.0.txt

Hostname:           x3-01-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-15-23+13:45:54

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected_perbase.bed.gz

show first few lines of file
chr11	4000000	4000001
chr11	4000001	4000002
chr11	4000002	4000003
chr11	4000003	4000004
chr11	4000004	4000005

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/fragment_count/OL13_20220512.hg38.norm.Log2FC.mean.stranded_pos.bed

show first few lines of file
chr11	61787329	61787528	chr11:61554801-61555000	-0.8	+
chr11	61787334	61787533	chr11:61554806-61555005	-0.784	+
chr11	61787339	61787538	chr11:61554811-61555010	-0.393	+
chr11	61787344	61787543	chr11:61554816-61555015	-0.493	+
chr11	61787349	61787548	chr11:61554821-61555020	-0.589	+

++++++++++++++++++++++++++++++++++++++
Output:  /data/reddylab/Kuei/out/proj_combeffect_enco

In [8]:
cat ${FD_LOG}/coverage_perbase_TMPRA_log2fc.1.txt

Hostname:           x3-01-2.genome.duke.edu
Slurm Array Index:  1
Time Stamp:         03-15-23+13:45:54

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected_perbase.bed.gz

show first few lines of file
chr11	4000000	4000001
chr11	4000001	4000002
chr11	4000002	4000003
chr11	4000003	4000004
chr11	4000004	4000005

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/fragment_count/OL43_20221003.hg38.norm.Log2FC.mean.stranded_pos.bed

show first few lines of file
chr8	126735901	126736100	chr8:126735901-126736100	3.197	+
chr8	126735951	126736150	chr8:126735951-126736150	2.74	+
chr8	126736001	126736200	chr8:126736001-126736200	2.963	+
chr8	126736051	126736250	chr8:126736051-126736250	0.494	+
chr8	126736101	126736300	chr8:126736101-126736300	0.856	+

++++++++++++++++++++++++++++++++++++++
Output:  /data/reddylab/Kuei/out/proj_combeffec

In [9]:
cat ${FD_LOG}/coverage_perbase_TMPRA_input_output.0.txt

Hostname:           x2-08-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-15-23+13:46:25

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected_perbase.bed.gz

show first few lines of file
chr11	4000000	4000001
chr11	4000001	4000002
chr11	4000002	4000003
chr11	4000003	4000004
chr11	4000004	4000005

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/fragment_count/OL13_20220512.hg38.norm.Input.mean.stranded_pos.bed

show first few lines of file
chr11	61787329	61787528	chr11:61554801-61555000	541.459	+
chr11	61787334	61787533	chr11:61554806-61555005	610.445	+
chr11	61787339	61787538	chr11:61554811-61555010	649.419	+
chr11	61787344	61787543	chr11:61554816-61555015	564.527	+
chr11	61787349	61787548	chr11:61554821-61555020	572.142	+

++++++++++++++++++++++++++++++++++++++
Output:  /data/reddylab/Kuei/out/proj_combeffec

In [10]:
cat ${FD_LOG}/coverage_perbase_TMPRA_input_output.59.txt

Hostname:           x3-01-4.genome.duke.edu
Slurm Array Index:  59
Time Stamp:         03-15-23+13:46:25

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected_perbase.bed.gz

show first few lines of file
chr11	4000000	4000001
chr11	4000001	4000002
chr11	4000002	4000003
chr11	4000003	4000004
chr11	4000004	4000005

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/fragment_count/OL45_20220927.hg38.raw.Output.rep4.stranded_pos.bed

show first few lines of file
chr11	4505501	4505700	chr11:4505501-4505700	714	+
chr11	4505601	4505800	chr11:4505601-4505800	395	+
chr11	4505701	4505900	chr11:4505701-4505900	291	+
chr11	4505801	4506000	chr11:4505801-4506000	406	+
chr11	4505901	4506100	chr11:4505901-4506100	179	+

++++++++++++++++++++++++++++++++++++++
Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMP